In [1]:
import os
import sys

# Move the working directory and change directory to the parent folder
parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)
os.chdir(parent_dir)

In [2]:
import asyncio
import pandas as pd
from datetime import datetime
from dotenv import load_dotenv
from IPython.display import display

from src.logger_initialization import initialize_logger
from src.utils import create_table, process_item_async
from graphs.table_graph import one_metrics_graph
from graphs.websearch_tool_graph import one_metrics_graph_tool

load_dotenv()
logger = initialize_logger("table_filling_experiment")

2025-12-29 16:28:41 - utils_logger - INFO - Logger 'utils_logger' initialized with log file: logs\utils_2025-12-29.log
2025-12-29 16:28:53 - parsing_utils_logger - INFO - Logger 'parsing_utils_logger' initialized with log file: logs\parsing_utils_2025-12-29.log
2025-12-29 16:28:54 - table_graph_logger - INFO - Logger 'table_graph_logger' initialized with log file: logs\table_graph_2025-12-29.log
2025-12-29 16:28:54 - websearch_tool_graph_logger - INFO - Logger 'websearch_tool_graph_logger' initialized with log file: logs\websearch_tool_graph_2025-12-29.log
2025-12-29 16:28:54 - table_filling_experiment_logger - INFO - Logger 'table_filling_experiment_logger' initialized with log file: logs\table_filling_experiment_2025-12-29.log


## Input: Companies and Metrics


In [3]:
parsed_data = pd.read_csv("data/european_company_metrics_2025.csv")

In [4]:
parsed_data

,Company,Ticker,country,city,employees,revenue,market_cap,ceo,error,size
0,HelloFresh,HFG.DE,Germany,Berlin,21783,7020299776,934806272,CEO,NaN,small
1,Fraport,FRA.DE,Germany,Frankfurt am Main,19843,4444000256,6435057152,chief executive officer|CEO,NaN,small
2,Zalando,ZAL.DE,Germany,Berlin,16117,11574599680,6529229312,NaN,NaN,small
3,Delivery Hero,DHER.DE,Germany,Berlin,43986,13401900032,6645038080,NaN,no CEO info,small
4,Telefónica,TEF.MC,Spain,Madrid,100870,41620000768,19604779008,Marc Murtra,NaN,small
5,Publicis Groupe,PUB.PA,France,Paris,111850,16862999552,22398087168,Arthur Sadoun,NaN,small
6,Leonardo (company),LDO.MI,Italy,Rome,62012,19131000832,27973961728,chairman,NaN,small
7,Ahold Delhaize,AD.AS,Netherlands,Zaandam,390000,92137996288,30924240896,Frans Muller,NaN,small
8,Orange Group,ORA.PA,France,Issy-les-Moulineaux,126909,40274001920,37526134784,Christel Heydemann,NaN,small
9,Adyen,ADYEN.AS,Netherlands,Amsterdam,4568,2191802880,43338702848,Chief executive officer|CEO,NaN,small


In [5]:
companies_list = parsed_data["Company"].tolist()

In [8]:
# Enter your metrics here (as a list)
metrics_list = ["CEO in 2024", "Revenue in 2024", "Gross margin in 2024"]

# Output file name (will be saved incrementally)
output_file = f"table_results_self_{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx"
print(f"Results will be saved to: {output_file}")

Results will be saved to: table_results_self_20251229_163046.xlsx


## Process and Save Incrementally

This will process each (company, metric) combination and save the table after each result.


In [9]:
async def process_and_save_incremental(
    companies_list, metrics_list, one_metrics_graph, output_file
):
    """
    Process all combinations and save the table incrementally after each result.
    This ensures we have partial results even if the process fails.
    """
    # Create all combinations
    combinations_list = [
        (company, metric) for company in companies_list for metric in metrics_list
    ]

    # Create configs for each combination
    configs = [
        {"configurable": {"thread_id": f"{company}_{metric}_{i}"}}
        for i, (company, metric) in enumerate(combinations_list)
    ]

    # Initialize results list
    results = []

    # Process each combination one by one and save after each
    total = len(combinations_list)
    for idx, (item, config) in enumerate(zip(combinations_list, configs), 1):
        company, metric = item
        print(f"\n[{idx}/{total}] Processing: {company} - {metric}")

        try:
            # Process the item
            result = await process_item_async(item, one_metrics_graph, config)
            results.append(result)

            # Create/update the table with current results
            df = create_table(results)

            # Save to Excel
            df.to_excel(output_file, index=False)
            print(
                f"✓ Saved progress to {output_file} ({len(results)}/{total} completed)"
            )

        except Exception as e:
            logger.error(f"Error processing {company} with {metric}: {e}")
            # Still add a result with error info
            results.append(
                {
                    "company_name": company,
                    "generation": {"error": str(e)},
                    "metrics": metric,
                }
            )
            # Save partial results even on error
            df = create_table(results)
            df.to_excel(output_file, index=False)
            print(f"✗ Error occurred, but saved partial results to {output_file}")

    print(f"\n✓ Processing complete! Final results saved to {output_file}")
    return results, df

In [10]:
import time

# Calculate time of completion
start_time = time.time()

# Run the async function
results, final_df = await process_and_save_incremental(
    companies_list, metrics_list, one_metrics_graph, output_file
)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"\nTotal processing time: {elapsed_time:.2f} seconds")



[1/60] Processing: HelloFresh - CEO in 2024
2025-12-29 16:30:49 - table_graph_logger - INFO - Thread: HelloFresh_CEO in 2024_0 - Web search
2025-12-29 16:30:49 - parsing_utils_logger - INFO - Start exa search with query: Find CEO in 2024 for HelloFresh
2025-12-29 16:30:55 - parsing_utils_logger - INFO - Exa search completed with 15 results
2025-12-29 16:30:55 - parsing_utils_logger - INFO - Found 15 new documents: ['https://www.linkedin.com/in/assafronen', 'https://www.eqs-news.com/news/corporate/hellofresh-se-announces-intention-of-co-founder-and-ceo-international-thomas-griesel-to-depart/e07dbf03-9f2c-4c5a-8de6-120c2daaecc5_en', 'https://en.wikipedia.org/wiki/HelloFresh', 'https://www.esmmagazine.com/technology/hellofresh-co-founder-thomas-griesel-steps-down-295777', 'https://tracxn.com/d/companies/hellofresh/__6Sz4mCoqD_LALvi1RL3is1YtB3f2eGhUx7F8yjDnwsI', 'https://careers.hellofresh.com/global/en/france', 'https://grizzlyreports.com/hfg1/', 'https://www.investing.com/news/transcrip

Processing documents: 100%|██████████| 15/15 [00:00<00:00, 29945.05it/s]

2025-12-29 16:31:01 - parsing_utils_logger - INFO - Processed 15 documents
2025-12-29 16:31:01 - table_graph_logger - INFO - Thread: HelloFresh_CEO in 2024_0 - Exa search done
2025-12-29 16:31:01 - parsing_utils_logger - INFO - ---ADD CREDIBILITY TO DOCUMENTS---
2025-12-29 16:31:01 - parsing_utils_logger - INFO - Adding credibility to web document -- https://www.linkedin.com/in/assafronen
2025-12-29 16:31:01 - parsing_utils_logger - INFO - Adding credibility to web document -- https://www.eqs-news.com/news/corporate/hellofresh-se-announces-intention-of-co-founder-and-ceo-international-thomas-griesel-to-depart/e07dbf03-9f2c-4c5a-8de6-120c2daaecc5_en
2025-12-29 16:31:01 - parsing_utils_logger - INFO - Adding credibility to web document -- https://en.wikipedia.org/wiki/HelloFresh
2025-12-29 16:31:01 - parsing_utils_logger - INFO - Adding credibility to web document -- https://www.esmmagazine.com/technology/hellofresh-co-founder-thomas-griesel-steps-down-295777
2025-12-29 16:31:01 - parsin

2025-12-29 16:31:04 - table_graph_logger - INFO - Thread: HelloFresh_CEO in 2024_0 - Credibility done
2025-12-29 16:31:04 - table_graph_logger - INFO - Thread: HelloFresh_CEO in 2024_0 - Creating retriever in memory...
2025-12-29 16:31:10 - parsing_utils_logger - INFO - Adding documents to the inmemory vectorstore
2025-12-29 16:31:13 - parsing_utils_logger - INFO - Documents added to the inmemory vectorstore
2025-12-29 16:31:13 - table_graph_logger - INFO - Thread: HelloFresh_CEO in 2024_0 - Retrieving documents from the vectorstore...
2025-12-29 16:31:14 - table_graph_logger - INFO - Thread: HelloFresh_CEO in 2024_0 - Generate
2025-12-29 16:31:26 - table_graph_logger - INFO - Thread: HelloFresh_CEO in 2024_0 - Generation: {'comment': 'HelloFresh’s CEO in 2024 was Dominik Richter. (According to HelloFresh’s official press release “HelloFresh SE Announces Intention of Co-Founder and CEO International, Thomas Griesel, to Depart,” which quotes “said Dominik Richter, co-founder and CEO of 

Processing documents: 100%|██████████| 15/15 [00:00<00:00, 35266.01it/s]

...

2025-12-29 17:41:38 - parsing_utils_logger - INFO - Processed 15 documents
2025-12-29 17:41:38 - table_graph_logger - INFO - Thread: LVMH_Revenue in 2024_58 - Exa search done
2025-12-29 17:41:38 - parsing_utils_logger - INFO - ---ADD CREDIBILITY TO DOCUMENTS---
2025-12-29 17:41:38 - parsing_utils_logger - INFO - Adding credibility to web document -- https://www.lvmh.com/en/publications/lvmh-achieves-a-solid-performance-despite-an-unfavorable-global-economic-environment
2025-12-29 17:41:38 - parsing_utils_logger - INFO - Adding credibility to web document -- https://www.fashiondive.com/news/lvmh-fiscal-2024-revenue-profits-down-luxury/738547/
2025-12-29 17:41:38 - parsing_utils_logger - INFO - Adding credibility to web document -- https://finance.yahoo.com/news/lvmh-generated-88-billion-revenue-000000860.html
2025-12-29 17:41:38 - parsing_utils_logger - INFO - Adding credibility to web document -- https://thepinklookbook.com/lvmh-2024-financial-results/
2025-12-29 17:41:38 - parsing_uti

2025-12-29 17:41:40 - table_graph_logger - INFO - Thread: LVMH_Revenue in 2024_58 - Credibility done
2025-12-29 17:41:40 - table_graph_logger - INFO - Thread: LVMH_Revenue in 2024_58 - Creating retriever in memory...
2025-12-29 17:41:50 - parsing_utils_logger - INFO - Adding documents to the inmemory vectorstore
2025-12-29 17:42:05 - parsing_utils_logger - INFO - Documents added to the inmemory vectorstore
2025-12-29 17:42:05 - table_graph_logger - INFO - Thread: LVMH_Revenue in 2024_58 - Retrieving documents from the vectorstore...
2025-12-29 17:42:06 - table_graph_logger - INFO - Thread: LVMH_Revenue in 2024_58 - Generate
2025-12-29 17:42:25 - table_graph_logger - INFO - Thread: LVMH_Revenue in 2024_58 - Generation: {'comment': 'LVMH’s reported revenue for 2024 is €84,683 million (i.e., €84.683 billion). This figure appears in LVMH’s official Investors and Analysts “Key Figures” income statement, which lists Revenue for 2022, 2023, and 2024 as 79,184; 86,153; and 84,683 respectively,

Processing documents: 100%|██████████| 15/15 [00:00<00:00, 50051.36it/s]

2025-12-29 17:42:49 - parsing_utils_logger - INFO - Processed 15 documents
2025-12-29 17:42:49 - table_graph_logger - INFO - Thread: LVMH_Gross margin in 2024_59 - Exa search done
2025-12-29 17:42:49 - parsing_utils_logger - INFO - ---ADD CREDIBILITY TO DOCUMENTS---
2025-12-29 17:42:49 - parsing_utils_logger - INFO - Adding credibility to web document -- https://www.lvmh.com/en/investors/investors-and-analysts
2025-12-29 17:42:49 - parsing_utils_logger - INFO - Adding credibility to web document -- https://finbox.com/DB:MOH/explorer/gp_margin/
2025-12-29 17:42:49 - parsing_utils_logger - INFO - Adding credibility to web document -- https://global.morningstar.com/en-ea/investments/stocks/0P0000G5GV/key-metrics?exchange=XMIL&ticker=1MC
2025-12-29 17:42:49 - parsing_utils_logger - INFO - Adding credibility to web document -- https://www.alphaspread.com/security/par/mc/profitability/ratio/gross-margin
2025-12-29 17:42:49 - parsing_utils_logger - INFO - Adding credibility to web document --

2025-12-29 17:42:50 - table_graph_logger - INFO - Thread: LVMH_Gross margin in 2024_59 - Credibility done
2025-12-29 17:42:50 - table_graph_logger - INFO - Thread: LVMH_Gross margin in 2024_59 - Creating retriever in memory...
2025-12-29 17:42:55 - parsing_utils_logger - INFO - Adding documents to the inmemory vectorstore
2025-12-29 17:43:04 - parsing_utils_logger - INFO - Documents added to the inmemory vectorstore
2025-12-29 17:43:04 - table_graph_logger - INFO - Thread: LVMH_Gross margin in 2024_59 - Retrieving documents from the vectorstore...
2025-12-29 17:43:04 - table_graph_logger - INFO - Thread: LVMH_Gross margin in 2024_59 - Generate
2025-12-29 17:43:20 - table_graph_logger - INFO - Thread: LVMH_Gross margin in 2024_59 - Generation: {'comment': 'Gross margin for LVMH in 2024 was €56,765 million, equivalent to 67% of revenue. This figure is reported directly in LVMH’s official “Key figures” page and matches the consolidated income statement published with the 2024 consolidated

In [ ]:
# 3840 seconds for gpt websearch

In [ ]:
# 4359.85 for self

## Display Final Results


In [ ]:
# Display the final table
display(final_df)

# Show summary
print(f"\nSummary:")
print(f"Total combinations processed: {len(results)}")
print(f"Companies: {len(final_df['company_name'].unique())}")
print(f"Metrics: {len([col for col in final_df.columns if col.endswith(' value')])}")
print(f"Output file: {output_file}")